In [1]:
import torch
from sklearn.datasets import fetch_openml

from neural_blueprints.architectures import MLP
from neural_blueprints.config.architectures import MLPConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.config.components.composite.projections.input import TabularInputProjectionConfig
from neural_blueprints.config.components.composite.projections.output import TabularOutputProjectionConfig
from neural_blueprints.utils import Trainer, infer_types, accuracy
from neural_blueprints.preprocess import TabularPreprocessor
from neural_blueprints.datasets import MaskedTabularDataset, TabularSingleLabelDataset

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [2]:
data = fetch_openml(name="adult", version=2, as_frame=True)
X = data.data
y = data.target

data = X.copy()
data['income'] = y

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)
data.head()

2025-12-23 17:14:08,987 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2025-12-23 17:14:08,987 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.109589,4,0.145129,2,14,5,7,4,3,2,0.000000,0.0,0.397959,39,1
1,0.287671,4,0.052451,12,16,3,5,1,5,2,0.000000,0.0,0.500000,39,1
2,0.150685,2,0.219649,8,4,3,11,1,5,2,0.000000,0.0,0.397959,39,2
3,0.369863,4,0.100153,16,2,3,7,1,3,2,0.076881,0.0,0.397959,39,2
4,0.013699,0,0.061708,16,2,5,0,4,5,1,0.000000,0.0,0.295918,39,1


### Income Inference Accuracy

In [4]:
dataset = TabularSingleLabelDataset(
    data=data,
    label_column='income',              # Specify the label column for single-label classification
    discrete_features=discrete_features,
    continuous_features=continuous_features
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [5]:
# Define model configuration
mlp_config = MLPConfig(
    hidden_dims=[128, 64, 32, 16],
    output_dim=3,
    normalization="batchnorm1d",
    activation='gelu',
    dropout_p=0.1,
    final_activation="softmax",
    input_projection=TabularInputProjectionConfig(
        cardinalities=dataset.cardinalities,
        hidden_dims=[64, 32],
        output_dim=[len(dataset.cardinalities)*16],
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.1
    )
)

# Initialize model
model = MLP(mlp_config)
model.blueprint()

2025-12-23 16:30:28,115 - neural_blueprints.architectures.mlp - INFO - Using input projection: TabularInputProjection


Sequential(
  (0): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropo

MLPConfig(input_dim=224, hidden_dims=[128, 64, 32, 16], output_dim=3, normalization='batchnorm1d', activation='gelu', dropout_p=0.1, final_activation='softmax', input_projection=TabularInputProjectionConfig(cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42], hidden_dims=[64, 32], output_dim=[224], dropout_p=0.1, normalization='batchnorm1d', activation='gelu'), output_projection=None)

In [6]:
trainer = Trainer(
    config=TrainerConfig(
        optimizer="adam",
        criterion="cross_entropy",
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=128,
        early_stopping_patience=5,
        save_weights_path="./models/mlp_adult.pth"
    ),
    model= model
)

# Train the model
trainer.train(train_dataset, val_dataset, epochs=20)

2025-12-23 16:30:28,450 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:   5%|▌         | 1/20 [00:02<00:46,  2.45s/epoch]

Epoch 1/20, Training Loss: 0.9645, Validation Loss: 0.9315


Training Epochs:  10%|█         | 2/20 [00:04<00:44,  2.45s/epoch]

...

Training Epochs:  60%|██████    | 12/20 [00:29<00:19,  2.46s/epoch]

Epoch 12/20, Training Loss: 0.9242, Validation Loss: 0.9289


Training Epochs:  65%|██████▌   | 13/20 [00:31<00:17,  2.46s/epoch]

Epoch 13/20, Training Loss: 0.9245, Validation Loss: 0.9266


2025-12-23 16:31:02,764 - neural_blueprints.utils.trainer - INFO - No improvement in validation loss for 5 consecutive epochs. Early stopping at epoch 14.
Training Epochs:  65%|██████▌   | 13/20 [00:34<00:18,  2.64s/epoch]
2025-12-23 16:31:02,764 - neural_blueprints.utils.trainer - INFO - Training completed in 34.31 seconds.
2025-12-23 16:31:02,764 - neural_blueprints.utils.trainer - INFO - Best validation loss: 9.2586e-01


In [7]:
X = torch.tensor(val_dataset[:][0])
y = val_dataset[:][1]
with torch.no_grad():
    y_pred = model(X)
    y_pred = y_pred.argmax(dim=1)
print(f"Predictions: {y_pred[:5]}, \n Ground Truth: {y[:5]}")
acc = accuracy(y_pred, y)
print(f"Validation Accuracy: {acc:.4f}")

/var/folders/5p/bjh6crbn6fv0vtxy5sqmmxh00000gn/T/ipykernel_5299/3930248022.py:1: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).



Predictions: tensor([1, 1, 1, 2, 1]), 
 Ground Truth: tensor([1, 1, 1, 1, 1])
Validation Accuracy: 0.8442


### Masked Dataset Inference Accuracy

In [4]:
dataset = MaskedTabularDataset(
    data = data,
    discrete_features = discrete_features,
    continuous_features = continuous_features,
    mask_prob=0.35
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [6]:
# Define model configuration
mlp_config = MLPConfig(
    hidden_dims=[128, 64, 32, 16],
    normalization="batchnorm1d",
    activation='gelu',
    dropout_p=0.1,
    final_activation="relu",
    input_projection=TabularInputProjectionConfig(
        cardinalities=dataset.cardinalities,
        hidden_dims=[64, 32],
        output_dim=[len(dataset.cardinalities) * 16],
        normalization="batchnorm1d",
        activation="gelu",
        dropout_p=0.1
    ),
    output_projection=TabularOutputProjectionConfig(
        cardinalities=dataset.cardinalities,
        input_dim=[len(dataset.cardinalities)*16],
        hidden_dims=[8],
        activation="gelu",
        normalization="batchnorm1d",
        dropout_p=0.1
    )
)

# Initialize model
model = MLP(mlp_config)
model.blueprint()

2025-12-23 17:14:31,746 - neural_blueprints.architectures.mlp - INFO - Using input projection: TabularInputProjection
2025-12-23 17:14:31,753 - neural_blueprints.architectures.mlp - INFO - Using output projection: TabularOutputProjection


Sequential(
  (0): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropo

MLPConfig(input_dim=240, hidden_dims=[128, 64, 32, 16], output_dim=240, normalization='batchnorm1d', activation='gelu', dropout_p=0.1, final_activation='relu', input_projection=TabularInputProjectionConfig(cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], hidden_dims=[64, 32], output_dim=[240], dropout_p=0.1, normalization='batchnorm1d', activation='gelu'), output_projection=TabularOutputProjectionConfig(cardinalities=[1, 9, 1, 16, 16, 7, 15, 6, 5, 2, 1, 1, 1, 42, 2], input_dim=[240], hidden_dims=[8], activation='gelu', normalization='batchnorm1d', dropout_p=0.1))

In [7]:
trainer = Trainer(
    config=TrainerConfig(
        optimizer="adam",
        criterion="mixed_type_reconstruction_loss",
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=128,
        early_stopping_patience=5,
        save_weights_path="./models/mlp_adult.pth"
    ),
    model= model
)

# Train the model
trainer.train(train_dataset, val_dataset, epochs=20)

2025-12-23 17:14:33,980 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:   5%|▌         | 1/20 [00:04<01:18,  4.15s/epoch]

Epoch 1/20, Training Loss: 219.1769, Validation Loss: 172.5390


Training Epochs:  10%|█         | 2/20 [00:08<01:14,  4.16s/epoch]

...

Epoch 18/20, Training Loss: 166.2095, Validation Loss: 164.8544


Training Epochs:  95%|█████████▌| 19/20 [01:18<00:04,  4.10s/epoch]

Epoch 19/20, Training Loss: 166.1493, Validation Loss: 164.8192


Training Epochs: 100%|██████████| 20/20 [01:22<00:00,  4.14s/epoch]
2025-12-23 17:15:56,699 - neural_blueprints.utils.trainer - INFO - Training completed in 82.72 seconds.
2025-12-23 17:15:56,699 - neural_blueprints.utils.trainer - INFO - Best validation loss: 1.6480e+02


Epoch 20/20, Training Loss: 166.1198, Validation Loss: 164.8040


In [8]:
X = val_dataset[:][0]
y = val_dataset[:][1]
mask = val_dataset[:][2]
with torch.no_grad():
    y_pred = model(x=X)

dis_accuracy = 0
cont_accuracy = 0
for column_idx, column_name in enumerate(data.columns):
    print(f"\nFeature Column {column_name}:")
    predicted_attributes = y_pred[column_idx]      # shape: (batch_size, num_classes)
    targets = y[:, column_idx]                     # shape: (batch_size,)

    feature_mask = mask[:, column_idx]                  # shape: (batch_size,)
    predicted_attributes = predicted_attributes[feature_mask]
    if predicted_attributes.size(1) > 1:
        predicted_attributes = predicted_attributes.softmax(dim=-1).argmax(dim=-1).cpu().numpy()
    else:
        predicted_attributes = predicted_attributes.squeeze(-1).cpu().numpy()
    targets = targets[feature_mask].cpu().numpy()

    print("Predicted attribute values:", predicted_attributes[:5])
    print("True attribute values:", targets[:5])

    accuracy_value = accuracy(torch.tensor(predicted_attributes), torch.tensor(targets))
    print(f"Accuracy: {accuracy_value:.4f}")
    if column_name in discrete_features:
        dis_accuracy += accuracy_value
    else:
        cont_accuracy += accuracy_value

avg_dis_accuracy = dis_accuracy / len(discrete_features) if len(discrete_features) > 0 else 0
avg_cont_accuracy = cont_accuracy / len(continuous_features) if len(continuous_features) > 0 else 0
print(f"\nAverage Discrete Accuracy: {avg_dis_accuracy:.4f}")
print(f"Average Continuous Accuracy: {avg_cont_accuracy:.4f}")
avg_accuracy = (dis_accuracy + cont_accuracy) / len(data.columns)
print(f"Overall Average Accuracy: {avg_accuracy:.4f}")


Feature Column age:
Predicted attribute values: [0.3205626  0.2902071  0.3127687  0.37595677 0.29728818]
True attribute values: [0.42465752 0.24657534 0.6849315  0.46575344 0.1780822 ]
Accuracy: 0.2362

Feature Column workclass:
Predicted attribute values: [4 4 4 4 4]
True attribute values: [4. 4. 4. 6. 2.]
Accuracy: 0.6589

Feature Column fnlwgt:
Predicted attribute values: [6.2287614e-07 1.9948704e-06 8.6632622e-07 9.6698034e-07 3.2716238e-07]
True attribute values: [0.18691577 0.12683587 0.14347734 0.02860468 0.02839766]
Accuracy: 0.1317

Feature Column education:
Predicted attribute values: [10 16 12 16 12]
True attribute values: [10. 13. 10.  3. 12.]
Accuracy: 0.5809

Feature Column education-num:
Predicted attribute values: [16  2  2  2  2]
True attribute values: [16.  2.  2. 13.  2.]
Accuracy: 0.5818

Feature Column marital-status:
Predicted attribute values: [3 3 5 3 5]
True attribute values: [3. 3. 3. 3. 5.]
Accuracy: 0.7209

Feature Column occupation:
Predicted attribute val